## <u>2. Datenaufbereitung Previous Application


*Vorgehensweise kategorische Variablen:*
- Gruppierung der Variablen
- Löschung von kategorischen Variablen mit weniger als 5pP relativer Anteilsdifferenz zwischen Paybacks und Defaults
- Löschung von Variablen mit mehr als 60% fehlenden Daten

## Initialisierung

In [1]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from sklearn.linear_model import LogisticRegression

from IPython.display import display, Markdown

In [2]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [3]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
pa = pd.read_csv(DATASET_DIR / "previous_application.csv")
pos = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "pos.csv")
ip = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "ip.csv")
cc = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "cc.csv")
description = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [4]:
pa = pa.set_index("SK_ID_PREV")
pos = pos.set_index("SK_ID_PREV")
ip = ip.set_index("SK_ID_PREV")
cc = cc.set_index("SK_ID_PREV")

In [5]:
des = description.loc[description['Table']=="previous_application.csv", "Row":"Special"]

In [6]:
des

,Row,Description,Special
176,SK_ID_PREV,"ID of previous credit in Home credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loan applications in Home Credit, previous application could, but not necessarily have to lead to credit)",hashed
177,SK_ID_CURR,ID of loan in our sample,hashed
178,NAME_CONTRACT_TYPE,"Contract product type (Cash loan, consumer loan [POS] ,...) of the previous application",NaN
179,AMT_ANNUITY,Annuity of previous application,NaN
180,AMT_APPLICATION,For how much credit did client ask on the previous application,NaN
181,AMT_CREDIT,"Final credit amount on the previous application. This differs from AMT_APPLICATION in a way that the AMT_APPLICATION is the amount for which the client initially applied for, but during our approval process he could have received different amount - AMT_CREDIT",NaN
182,AMT_DOWN_PAYMENT,Down payment on the previous application,NaN
183,AMT_GOODS_PRICE,Goods price of good that client asked for (if applicable) on the previous application,NaN
184,WEEKDAY_APPR_PROCESS_START,On which day of the week did the client apply for previous application,NaN
185,HOUR_APPR_PROCESS_START,Approximately at what day hour did the client apply for the previous application,rounded


In [7]:
pa.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Spalten die innerhalb der Aufbereitung nicht verändert werden können
skip = ["TARGET", "SK_ID_CURR", "SK_ID_PREV"]

In [9]:
# nominale und metrische Spalten
n_heads = [element for element in pa.columns if pa[element].dtype.name == "object"]
n_heads += ["NFLAG_LAST_APPL_IN_DAY", "NFLAG_INSURED_ON_APPROVAL"]
o_heads = ["HOUR_APPR_PROCESS_START", "SELLERPLACE_AREA"]
m_heads = [element for element in pa.columns if element not in n_heads and element not in o_heads]

## <u> kategorische Variablen

In [10]:
# die ordinalen Variablen werden beide nicht übernommen

In [11]:
df = pa[["SK_ID_CURR"] + n_heads].copy()

In [12]:
result = pd.DataFrame(index=pa.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

In [13]:
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL
SK_ID_PREV,,,,,,,,,,,,,,,,,,,
2030495,271877,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,Connectivity,middle,POS mobile with interest,1,0.0
2802425,108129,Cash loans,THURSDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,XNA,low_action,Cash X-Sell: low,1,1.0
2523466,122040,Cash loans,TUESDAY,Y,XNA,Approved,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,high,Cash X-Sell: high,1,1.0
2819243,176158,Cash loans,MONDAY,Y,XNA,Approved,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,middle,Cash X-Sell: middle,1,1.0
1784265,202054,Cash loans,THURSDAY,Y,Repairs,Refused,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,XNA,high,Cash Street: high,1,NaN


### Aufsummierung aller historischen Kredite nach Kategorie

In [14]:
for cat in df.columns.values:
    if cat in skip:
        continue
    
    A = df[["SK_ID_CURR", cat]].groupby(by=["SK_ID_CURR", cat]).size().unstack(fill_value=0)
    A.columns = ["CNT_{cat}_{head}".format(cat=cat, head=head) for head in A.keys().values]
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

In [15]:
df = result
df.head()

,CNT_NAME_CONTRACT_TYPE_Cash loans,CNT_NAME_CONTRACT_TYPE_Consumer loans,CNT_NAME_CONTRACT_TYPE_Revolving loans,CNT_NAME_CONTRACT_TYPE_XNA,CNT_WEEKDAY_APPR_PROCESS_START_FRIDAY,CNT_WEEKDAY_APPR_PROCESS_START_MONDAY,CNT_WEEKDAY_APPR_PROCESS_START_SATURDAY,CNT_WEEKDAY_APPR_PROCESS_START_SUNDAY,CNT_WEEKDAY_APPR_PROCESS_START_THURSDAY,CNT_WEEKDAY_APPR_PROCESS_START_TUESDAY,CNT_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,CNT_FLAG_LAST_APPL_PER_CONTRACT_N,CNT_FLAG_LAST_APPL_PER_CONTRACT_Y,CNT_NAME_CASH_LOAN_PURPOSE_Building a house or an annex,CNT_NAME_CASH_LOAN_PURPOSE_Business development,CNT_NAME_CASH_LOAN_PURPOSE_Buying a garage,CNT_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,CNT_NAME_CASH_LOAN_PURPOSE_Buying a home,CNT_NAME_CASH_LOAN_PURPOSE_Buying a new car,CNT_NAME_CASH_LOAN_PURPOSE_Buying a used car,CNT_NAME_CASH_LOAN_PURPOSE_Car repairs,CNT_NAME_CASH_LOAN_PURPOSE_Education,CNT_NAME_CASH_LOAN_PURPOSE_Everyday expenses,CNT_NAME_CASH_LOAN_PURPOSE_Furniture,CNT_NAME_CASH_LOAN_PURPOSE_Gasification / water supply,CNT_NAME_CASH_LOAN_PURPOSE_Hobby,CNT_NAME_CASH_LOAN_PURPOSE_Journey,CNT_NAME_CASH_LOAN_PURPOSE_Medicine,CNT_NAME_CASH_LOAN_PURPOSE_Money for a third person,CNT_NAME_CASH_LOAN_PURPOSE_Other,CNT_NAME_CASH_LOAN_PURPOSE_Payments on other loans,CNT_NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment,CNT_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal,CNT_NAME_CASH_LOAN_PURPOSE_Repairs,CNT_NAME_CASH_LOAN_PURPOSE_Urgent needs,CNT_NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday,CNT_NAME_CASH_LOAN_PURPOSE_XAP,CNT_NAME_CASH_LOAN_PURPOSE_XNA,CNT_NAME_CONTRACT_STATUS_Approved,CNT_NAME_CONTRACT_STATUS_Canceled,CNT_NAME_CONTRACT_STATUS_Refused,CNT_NAME_CONTRACT_STATUS_Unused offer,CNT_NAME_PAYMENT_TYPE_Cash through the bank,CNT_NAME_PAYMENT_TYPE_Cashless from the account of the employer,CNT_NAME_PAYMENT_TYPE_Non-cash from your account,CNT_NAME_PAYMENT_TYPE_XNA,CNT_CODE_REJECT_REASON_CLIENT,CNT_CODE_REJECT_REASON_HC,CNT_CODE_REJECT_REASON_LIMIT,CNT_CODE_REJECT_REASON_SCO,CNT_CODE_REJECT_REASON_SCOFR,CNT_CODE_REJECT_REASON_SYSTEM,CNT_CODE_REJECT_REASON_VERIF,CNT_CODE_REJECT_REASON_XAP,CNT_CODE_REJECT_REASON_XNA,CNT_NAME_TYPE_SUITE_Children,CNT_NAME_TYPE_SUITE_Family,CNT_NAME_TYPE_SUITE_Group of people,CNT_NAME_TYPE_SUITE_Other_A,CNT_NAME_TYPE_SUITE_Other_B,"CNT_NAME_TYPE_SUITE_Spouse, partner",CNT_NAME_TYPE_SUITE_Unaccompanied,CNT_NAME_CLIENT_TYPE_New,CNT_NAME_CLIENT_TYPE_Refreshed,CNT_NAME_CLIENT_TYPE_Repeater,CNT_NAME_CLIENT_TYPE_XNA,CNT_NAME_GOODS_CATEGORY_Additional Service,CNT_NAME_GOODS_CATEGORY_Animals,CNT_NAME_GOODS_CATEGORY_Audio/Video,CNT_NAME_GOODS_CATEGORY_Auto Accessories,CNT_NAME_GOODS_CATEGORY_Clothing and Accessories,CNT_NAME_GOODS_CATEGORY_Computers,CNT_NAME_GOODS_CATEGORY_Construction Materials,CNT_NAME_GOODS_CATEGORY_Consumer Electronics,CNT_NAME_GOODS_CATEGORY_Direct Sales,CNT_NAME_GOODS_CATEGORY_Education,CNT_NAME_GOODS_CATEGORY_Fitness,CNT_NAME_GOODS_CATEGORY_Furniture,CNT_NAME_GOODS_CATEGORY_Gardening,CNT_NAME_GOODS_CATEGORY_Homewares,CNT_NAME_GOODS_CATEGORY_House Construction,CNT_NAME_GOODS_CATEGORY_Insurance,CNT_NAME_GOODS_CATEGORY_Jewelry,CNT_NAME_GOODS_CATEGORY_Medical Supplies,CNT_NAME_GOODS_CATEGORY_Medicine,CNT_NAME_GOODS_CATEGORY_Mobile,CNT_NAME_GOODS_CATEGORY_Office Appliances,CNT_NAME_GOODS_CATEGORY_Other,CNT_NAME_GOODS_CATEGORY_Photo / Cinema Equipment,CNT_NAME_GOODS_CATEGORY_Sport and Leisure,CNT_NAME_GOODS_CATEGORY_Tourism,CNT_NAME_GOODS_CATEGORY_Vehicles,CNT_NAME_GOODS_CATEGORY_Weapon,CNT_NAME_GOODS_CATEGORY_XNA,CNT_NAME_PORTFOLIO_Cards,CNT_NAME_PORTFOLIO_Cars,CNT_NAME_PORTFOLIO_Cash,CNT_NAME_PORTFOLIO_POS,CNT_NAME_PORTFOLIO_XNA,CNT_NAME_PRODUCT_TYPE_XNA,CNT_NAME_PRODUCT_TYPE_walk-in,CNT_NAME_PRODUCT_TYPE_x-sell,CNT_CHANNEL_TYPE_AP+ (Cash loan),CNT_CHANNEL_TYPE_Car dealer,CNT_CHANNEL_TYPE_Channel of corporate sales,CNT_CHANNEL_TYPE_Contact center,CNT_CHANNEL_TYPE_Country-wide,CNT_CHANNEL_TYPE_Credit and cash offices,CNT_CHANNEL_TYPE_Regional / Local,CNT_CHANNEL_TYPE_Stone,CNT_NAME_SELLER_INDUSTR

Kreditnehmer 271877 hat in seiner Kredithistorie von Home Credit 3 Konsumentenkredite beantragt, wovon 2 genehmigt und 1 abgelehnt wurde, etc. 

### Unterscheidbarkeit von mindestens 5pP einer Kategorie

In [16]:
target = app_train[["SK_ID_CURR", "TARGET"]]
target = target.set_index("SK_ID_CURR")

In [17]:
df = pd.merge(df, target, left_index=True, right_index=True)

In [18]:
ID_Payback = df[df["TARGET"] == 0].index.values
ID_Default = df[df["TARGET"] == 1].index.values

In [19]:
payback = df.loc[ID_Payback]
default = df.loc[ID_Default]

In [20]:
result = {
    "head" : [],
    "cat" : [],
    "payback" : [],
    "default" : [],
    "diff" : []
}

for head in df.columns.values:
    df1 = payback[head].value_counts().rename_axis(head).reset_index(name='payback')
    df2 = default[head].value_counts().rename_axis(head).reset_index(name='default')
    
    df1["payback"] = df1["payback"]/df1["payback"].sum()*100
    df2["default"] = df2["default"]/df2["default"].sum()*100
    
    df_ = df1.merge(df2, how="outer", on=head)
    
    df_["diff"] = (df_["default"]-df_["payback"])
    
    df_ = df_.sort_values("diff", ascending=False)
    
    for diff in df_["diff"]:
        if np.isnan(diff):
            continue
        if diff > 5 or diff < -5:
            row = df_.loc[df_["diff"] == diff]
            cat = row[head][row[head].index[0]]
            
            result["head"].append(head)
            result["cat"].append(cat)
            result["payback"].append(round(row["payback"].values[0],2))
            result["default"].append(round(row["default"].values[0],2))
            result["diff"].append(round(diff,2))

result = pd.DataFrame(result)
result.sort_values("diff", ascending=False)

,head,cat,payback,default,diff
8,CNT_NAME_YIELD_GROUP_low_normal,0,45.91,52.84,6.93
10,CNT_PRODUCT_COMBINATION_Cash X-Sell: low,0,76.94,82.46,5.51
4,CNT_NAME_CLIENT_TYPE_Refreshed,0,73.22,78.47,5.25
11,CNT_PRODUCT_COMBINATION_POS industry with interest,0,77.51,82.73,5.23
5,CNT_NAME_PORTFOLIO_Cards,0,66.92,60.89,-6.03
3,CNT_NAME_CLIENT_TYPE_New,0,19.04,12.89,-6.15
7,CNT_NAME_YIELD_GROUP_high,0,44.21,38.07,-6.15
9,CNT_PRODUCT_COMBINATION_Card Street,0,78.02,71.85,-6.17
0,CNT_NAME_CONTRACT_TYPE_Revolving loans,0,64.77,58.40,-6.37
2,CNT_CODE_REJECT_REASON_HC,0,77.91,70.69,-7.22


Defaults haben häufiger keine Kredite mit niedrigen oder mittleren Zinsen (hohe Zinsen), sie sind Neukunden, etc.

Paybacks wurden in der Vergangenheit nie abgelehnt, haben niedrige oder mittlere Zinsen, etc.

In [21]:
remove = [head for head in df.columns.values if head not in list(result["head"])  + skip]

In [22]:
len(remove)

135

In [23]:
df = df.drop(remove, axis=1)

In [24]:
df.head()

,CNT_NAME_CONTRACT_TYPE_Revolving loans,CNT_NAME_CONTRACT_STATUS_Refused,CNT_CODE_REJECT_REASON_HC,CNT_NAME_CLIENT_TYPE_New,CNT_NAME_CLIENT_TYPE_Refreshed,CNT_NAME_PORTFOLIO_Cards,CNT_NAME_PRODUCT_TYPE_walk-in,CNT_NAME_YIELD_GROUP_high,CNT_NAME_YIELD_GROUP_low_normal,CNT_PRODUCT_COMBINATION_Card Street,CNT_PRODUCT_COMBINATION_Cash X-Sell: low,CNT_PRODUCT_COMBINATION_POS industry with interest,TARGET
SK_ID_CURR,,,,,,,,,,,,,
271877,0,1,0,1,0,0,0,0,1,0,0,2,0
108129,1,0,0,1,0,1,0,1,2,0,2,2,0
122040,1,0,0,0,0,1,0,2,0,1,0,0,0
176158,0,15,14,0,2,0,4,7,2,0,0,0,0
202054,1,13,11,1,0,1,4,8,2,0,0,0,0


### Löschung der Spalten mit weniger als 40% ausgefüllten Daten

In [25]:
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in df.keys():
    if key in skip:
        continue
    rate = df[key].isna().sum() / len(df[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
result

,header,rate,des


In [26]:
df = df.drop(result.header.values, axis=1)

In den kategorischen Variablen gibt es keine Variablen mit einer Quote von +60% fehlender Daten

### Ergebnis

In [27]:
df = df.drop(["TARGET"], axis=1)

In [28]:
df.head()

,CNT_NAME_CONTRACT_TYPE_Revolving loans,CNT_NAME_CONTRACT_STATUS_Refused,CNT_CODE_REJECT_REASON_HC,CNT_NAME_CLIENT_TYPE_New,CNT_NAME_CLIENT_TYPE_Refreshed,CNT_NAME_PORTFOLIO_Cards,CNT_NAME_PRODUCT_TYPE_walk-in,CNT_NAME_YIELD_GROUP_high,CNT_NAME_YIELD_GROUP_low_normal,CNT_PRODUCT_COMBINATION_Card Street,CNT_PRODUCT_COMBINATION_Cash X-Sell: low,CNT_PRODUCT_COMBINATION_POS industry with interest
SK_ID_CURR,,,,,,,,,,,,
271877,0,1,0,1,0,0,0,0,1,0,0,2
108129,1,0,0,1,0,1,0,1,2,0,2,2
122040,1,0,0,0,0,1,0,2,0,1,0,0
176158,0,15,14,0,2,0,4,7,2,0,0,0
202054,1,13,11,1,0,1,4,8,2,0,0,0


### resultierende Variablen

In [29]:
result = {
    "head":[],
    "des":[]
}

HEADERS = ["NAME_CONTRACT_TYPE", "NAME_CONTRACT_STATUS", "CODE_REJECT_REASON", "NAME_CLIENT_TYPE", "NAME_PORTFOLIO", "NAME_PRODUCT_TYPE", "NAME_YIELD_GROUP", "PRODUCT_COMBINATION"]

for head in HEADERS:
    if head in skip:
        continue
    result["head"].append(head)
    result["des"].append(des[des["Row"] == head]["Description"])
    
result = pd.DataFrame(result)
result

,head,des
0,NAME_CONTRACT_TYPE,"178 Contract product type (Cash loan, consumer loan [POS] ,...) of the previous application Name: Description, dtype: object"
1,NAME_CONTRACT_STATUS,"193 Contract status (approved, cancelled, ...) of previous application Name: Description, dtype: object"
2,CODE_REJECT_REASON,"196 Why was the previous application rejected Name: Description, dtype: object"
3,NAME_CLIENT_TYPE,"198 Was the client old or new client when applying for the previous application Name: Description, dtype: object"
4,NAME_PORTFOLIO,"200 Was the previous application for CASH, POS, CAR, Name: Description, dtype: object"
5,NAME_PRODUCT_TYPE,"201 Was the previous application x-sell o walk-in Name: Description, dtype: object"
6,NAME_YIELD_GROUP,"206 Grouped interest rate into small medium and high of the previous application Name: Description, dtype: object"
7,PRODUCT_COMBINATION,"207 Detailed product combination of the previous application Name: Description, dtype: object"


### Speichern der kategorischen Werte

In [30]:
cats = df

## <u> metrische Variablen

In [31]:
ID_PA_POS = np.intersect1d(pa.index.values, pos.index.values)
ID_PA_IP = np.intersect1d(pa.index.values, ip.index.values)
ID_PA_CC = np.intersect1d(pa.index.values, cc.index.values)

# Ermittlung der Kredite ohne Kredithistorie
# -> PA's, die nicht in POS, IP oder CC sind

A = np.concatenate((ID_PA_POS,ID_PA_IP,ID_PA_CC)) 
B = pa.drop(A)

ID_PA_ONLY = B.index.values

In [32]:
# Erstellung der Unterdatensätze je nach Kredittyp

In [33]:
pa_pos = pa.loc[ID_PA_POS]
pa_ip = pa.loc[ID_PA_IP]
pa_cc = pa.loc[ID_PA_CC]
pa_only = pa.loc[ID_PA_ONLY]

In [34]:
pa_pos = pd.merge(pa_pos, pos, how="left", left_index=True, right_index=True)
pa_ip = pd.merge(pa_ip, ip, how="left", left_index=True, right_index=True)
pa_cc = pd.merge(pa_cc, cc, how="left", left_index=True, right_index=True)

### Löschung der Spalten mit weniger als 40% ausgefüllten Daten

In [35]:
# pa_pos

df = pa_pos
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in df.keys():
    if key in skip:
        continue
    rate = df[key].isna().sum() / len(df[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
pa_pos = df.drop(result.header.values, axis=1)
result

,header,rate,des
0,RATE_INTEREST_PRIMARY,99.396375,"190 Interest rate normalized on previous credit Name: Description, dtype: object"
1,RATE_INTEREST_PRIVILEGED,99.396375,"191 Interest rate normalized on previous credit Name: Description, dtype: object"


Der konkrete Zinssatz ist fast nie hinterlegt

In [36]:
# pa_ip

df = pa_ip
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in df.keys():
    if key in skip:
        continue
    rate = df[key].isna().sum() / len(df[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
pa_ip = df.drop(result.header.values, axis=1)
result

,header,rate,des
0,RATE_INTEREST_PRIMARY,99.024898,"190 Interest rate normalized on previous credit Name: Description, dtype: object"
1,RATE_INTEREST_PRIVILEGED,99.024898,"191 Interest rate normalized on previous credit Name: Description, dtype: object"


In [37]:
# pa_cc

df = pa_cc
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in df.keys():
    if key in skip:
        continue
    rate = df[key].isna().sum() / len(df[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
pa_cc = df.drop(result.header.values, axis=1)
result

,header,rate,des
0,AMT_DOWN_PAYMENT,99.141317,"182 Down payment on the previous application Name: Description, dtype: object"
1,RATE_DOWN_PAYMENT,99.141317,"189 Down payment rate normalized on previous credit Name: Description, dtype: object"
2,RATE_INTEREST_PRIMARY,100.000000,"190 Interest rate normalized on previous credit Name: Description, dtype: object"
3,RATE_INTEREST_PRIVILEGED,100.000000,"191 Interest rate normalized on previous credit Name: Description, dtype: object"
4,NAME_TYPE_SUITE,62.644055,"197 Who accompanied client when applying for the previous application Name: Description, dtype: object"


In [38]:
# pa_only

df = pa_only
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in df.keys():
    if key in skip:
        continue
    rate = df[key].isna().sum() / len(df[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
pa_only = df.drop(result.header.values, axis=1)
result

,header,rate,des
0,AMT_DOWN_PAYMENT,80.897874,"182 Down payment on the previous application Name: Description, dtype: object"
1,RATE_DOWN_PAYMENT,80.897874,"189 Down payment rate normalized on previous credit Name: Description, dtype: object"
2,RATE_INTEREST_PRIMARY,99.923179,"190 Interest rate normalized on previous credit Name: Description, dtype: object"
3,RATE_INTEREST_PRIVILEGED,99.923179,"191 Interest rate normalized on previous credit Name: Description, dtype: object"
4,NAME_TYPE_SUITE,68.265060,"197 Who accompanied client when applying for the previous application Name: Description, dtype: object"
5,DAYS_FIRST_DRAWING,99.181512,"208 Relative to application date of current application when was the first disbursement of the previous application Name: Description, dtype: object"
6,DAYS_FIRST_DUE,99.181512,"209 Relative to application date of current application when was the first due supposed to be of the previous application Name: Description, dtype: object"
7,DAYS_LAST_DUE_1ST_VERSION,99.181512,"210 Relative to application date of current application when was the first due of the previous application Name: Description, dtype: object"
8,DAYS_LAST_DUE,99.181512,"211 Relative to application date of current application when was the last due date of the previous application Name: Description, dtype: object"
9,DAYS_TERMINATION,99.181512,"212 Relative to application date of current application when was the expected termination of the previous application Name: Description, dtype: object"


## PA_ONLY

In [39]:
# PA_ONLY

In [40]:
m_heads_only = [head for head in m_heads if head in pa_only.columns.values]

In [41]:
df = pa_only[m_heads_only].copy()

In [42]:
df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,DAYS_DECISION,CNT_PAYMENT
SK_ID_PREV,,,,,,,
1784265,202054,31924.395,337500.0,404055.0,337500.0,-781,24.0
2315218,175704,NaN,0.0,0.0,NaN,-14,NaN
1656711,296299,NaN,0.0,0.0,NaN,-21,NaN
2367563,342292,NaN,0.0,0.0,NaN,-386,NaN
2579447,334349,NaN,0.0,0.0,NaN,-57,NaN


In [43]:
result = {
    "head":[],
    "des":[]
}

for head in df.columns.values:
    if head in skip:
        continue
    result["head"].append(head)
    result["des"].append(des[des["Row"] == head]["Description"])
    
result = pd.DataFrame(result)
result

,head,des
0,AMT_ANNUITY,"179 Annuity of previous application Name: Description, dtype: object"
1,AMT_APPLICATION,"180 For how much credit did client ask on the previous application Name: Description, dtype: object"
2,AMT_CREDIT,"181 Final credit amount on the previous application. This differs from AMT_APPLICATION in a way that the AMT_APPLICATION is the amount for which the client initially applied for, but during our approval process he could have received different amount - AMT_CREDIT Name: Description, dtype: object"
3,AMT_GOODS_PRICE,"183 Goods price of good that client asked for (if applicable) on the previous application Name: Description, dtype: object"
4,DAYS_DECISION,"194 Relative to current application when was the decision about previous application made Name: Description, dtype: object"
5,CNT_PAYMENT,"205 Term of previous credit at application of the previous application Name: Description, dtype: object"


Ergebnis: pa_only müssen komplett gelöscht werden, weil nicht nachvollziehbar ist welche Zahlungen wann erfolgen

## PA_POS

In [44]:
# pa_pos

In [45]:
df = pa_pos
df = df.drop(n_heads + o_heads, axis=1)
df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,POS_CNT_PAYMENTS_LEFT,POS_CNT_DPD
SK_ID_PREV,,,,,,,,,,,,,,,,
1000001,158271,6404.310,58905.000,65124.0,0.000,58905.000,0.000000,-299,12.0,365243.0,-268.0,62.0,-238.0,-233.0,0.0,0.0
1000002,101962,6264.000,39145.500,35230.5,3915.000,39145.500,0.108922,-1634,6.0,365243.0,-1600.0,-1450.0,-1510.0,-1501.0,0.0,0.0
1000003,252457,4951.350,47056.275,52641.0,4.275,47056.275,0.000088,-124,12.0,365243.0,-94.0,236.0,365243.0,365243.0,9.0,0.0
1000004,260094,3391.110,35144.370,30586.5,7032.870,35144.370,0.203603,-893,10.0,365243.0,-862.0,-592.0,-682.0,-672.0,0.0,0.0
1000005,176456,14713.605,123486.075,120307.5,12349.575,123486.075,0.101388,-1719,10.0,365243.0,-1688.0,-1418.0,-1418.0,-1415.0,0.0,0.0


In [46]:
result = pd.DataFrame(index=pa_pos.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

### Gruppierung

In [47]:
# count
A = df[["SK_ID_CURR"]]
A.reset_index(level=0, inplace=True)
A = A.groupby(by=["SK_ID_CURR"]).count()
A.columns = ["CNT_CREDITS"]
result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Anzahl der vergangenen POS Kredite

In [48]:
# sums
sum_heads = ["POS_CNT_PAYMENTS_LEFT", "POS_CNT_DPD"] + ["AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE", "RATE_DOWN_PAYMENT"]

for head in sum_heads:
    A = df[["SK_ID_CURR", head]].groupby(by=["SK_ID_CURR"]).sum()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Hat ein Kreditnehmer mehrere Kredite paralell werden deren Daten aufsummiert um die gesamte jährliche Zahlungshöhe, Kredithöhe etc zu ermitteln.

### Zusammenfügen und Speichern der PA_POS Daten

In [49]:
pa_pos = result

In [50]:
pa_pos = pd.merge(pa_pos, cats, how="left", left_index=True, right_index=True)

In [51]:
pa_pos = pd.merge(pa_pos, target, left_index=True, right_index=True) # Nur Kredite die auch in Application enthalten sind werden übernommen
pa_pos = pa_pos.drop(["TARGET"], axis=1)

In [52]:
pa_pos = pa_pos.add_prefix("PA_")

In [53]:
pa_pos.head()

,PA_CNT_CREDITS,PA_POS_CNT_PAYMENTS_LEFT,PA_POS_CNT_DPD,PA_AMT_ANNUITY,PA_AMT_APPLICATION,PA_AMT_CREDIT,PA_AMT_DOWN_PAYMENT,PA_AMT_GOODS_PRICE,PA_RATE_DOWN_PAYMENT,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_CODE_REJECT_REASON_HC,PA_CNT_NAME_CLIENT_TYPE_New,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_CNT_NAME_PORTFOLIO_Cards,PA_CNT_NAME_PRODUCT_TYPE_walk-in,PA_CNT_NAME_YIELD_GROUP_high,PA_CNT_NAME_YIELD_GROUP_low_normal,PA_CNT_PRODUCT_COMBINATION_Card Street,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low,PA_CNT_PRODUCT_COMBINATION_POS industry with interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
158271,1,0.0,0.0,6404.310,58905.000,65124.0,0.000,58905.000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
252457,1,9.0,0.0,4951.350,47056.275,52641.0,4.275,47056.275,0.000088,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
260094,9,70.0,0.0,97839.945,2023469.370,2294779.5,8765.370,2023469.370,0.312513,0.0,1.0,1.0,1.0,1.0,0.0,1.0,4.0,3.0,0.0,4.0,0.0
176456,1,0.0,0.0,14713.605,123486.075,120307.5,12349.575,123486.075,0.101388,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
256657,3,28.0,0.0,60139.575,1115806.500,1277523.0,78714.000,1115806.500,0.699443,2.0,7.0,5.0,1.0,0.0,2.0,5.0,0.0,2.0,1.0,0.0,0.0


In [54]:
pa_pos.to_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_pos.csv")

## PA_IP

In [55]:
# pa_ip

In [56]:
df = pa_ip
df = df.drop(n_heads + o_heads, axis=1)
df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,IP_PAYMENT_30,IP_DAYS_DIFF_30,IP_AMT_DIFF_30,IP_PAYMENT_60,IP_DAYS_DIFF_60,IP_AMT_DIFF_60,IP_PAYMENT_90,IP_DAYS_DIFF_90,IP_AMT_DIFF_90,IP_PAYMENT_120,IP_DAYS_DIFF_120,IP_AMT_DIFF_120,IP_PAYMENT_150,IP_DAYS_DIFF_150,IP_AMT_DIFF_150,IP_PAYMENT_180,IP_DAYS_DIFF_180,IP_AMT_DIFF_180
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000003,252457,4951.350,47056.275,52641.0,4.275,47056.275,0.000088,-124,12.0,365243.0,-94.0,236.0,365243.0,365243.0,-0.912871,0.0,0.0,0.912871,-15.0,0.0,0.912871,-17.0,0.0,0.912871,-14.0,0.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0
1000007,256657,11246.805,78570.000,62856.0,15714.000,78570.000,0.217818,-158,6.0,365243.0,-123.0,27.0,365243.0,365243.0,0.408248,-7.0,0.0,0.408248,-21.0,0.0,0.408248,-17.0,0.0,0.408248,-19.0,0.0,0.408248,-20.0,0.0,-2.041241,0.0,0.0
1000011,198678,92435.040,855000.000,879831.0,NaN,855000.000,NaN,-465,12.0,365243.0,-435.0,-105.0,-105.0,-101.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0,0.912056,-2.0,0.0,0.913278,1.0,0.0,0.913278,-1.0,0.0
1000017,310743,7002.720,67864.230,75024.0,4.230,67864.230,0.000061,-516,12.0,365243.0,-486.0,-156.0,-156.0,-151.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,2.041241,-5.0,0.0
1000018,394447,2250.000,45000.000,45000.0,NaN,45000.000,NaN,-176,0.0,-175.0,-130.0,365243.0,365243.0,365243.0,0.650875,0.0,0.0,0.814828,-1.0,0.0,-1.044958,-1.0,0.0,-0.867506,-2.0,0.0,-0.768616,-2.0,0.0,1.215377,0.0,0.0


In [57]:
result = pd.DataFrame(index=pa_ip.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

In [58]:
# count
A = df[["SK_ID_CURR"]]
A.reset_index(level=0, inplace=True)
A = A.groupby(by=["SK_ID_CURR"]).count()
A.columns = ["CNT_CREDITS"]
result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Anzahl der vergangenen Install Payments Kredite

In [59]:
ip.columns.values

array(['IP_PAYMENT_30', 'IP_DAYS_DIFF_30', 'IP_AMT_DIFF_30',
       'IP_PAYMENT_60', 'IP_DAYS_DIFF_60', 'IP_AMT_DIFF_60',
       'IP_PAYMENT_90', 'IP_DAYS_DIFF_90', 'IP_AMT_DIFF_90',
       'IP_PAYMENT_120', 'IP_DAYS_DIFF_120', 'IP_AMT_DIFF_120',
       'IP_PAYMENT_150', 'IP_DAYS_DIFF_150', 'IP_AMT_DIFF_150',
       'IP_PAYMENT_180', 'IP_DAYS_DIFF_180', 'IP_AMT_DIFF_180'],
      dtype=object)

In [60]:
# means
mean_heads = list(ip.columns.values)

for head in mean_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).mean()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Aus mehereren Zeitreihen wird der Durchschnitt gebildet, da die Daten normalisiert sind

In [61]:
# sums
sums_heads = ["AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE", "RATE_DOWN_PAYMENT"]

for head in sums_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).sum()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Hat ein Kreditnehmer mehrere Kredite paralell werden deren Daten aufsummiert um die gesamte jährliche Zahlungshöhe, Kredithöhe etc zu ermitteln.

In [62]:
result.head()

,CNT_CREDITS,IP_PAYMENT_30,IP_DAYS_DIFF_30,IP_AMT_DIFF_30,IP_PAYMENT_60,IP_DAYS_DIFF_60,IP_AMT_DIFF_60,IP_PAYMENT_90,IP_DAYS_DIFF_90,IP_AMT_DIFF_90,IP_PAYMENT_120,IP_DAYS_DIFF_120,IP_AMT_DIFF_120,IP_PAYMENT_150,IP_DAYS_DIFF_150,IP_AMT_DIFF_150,IP_PAYMENT_180,IP_DAYS_DIFF_180,IP_AMT_DIFF_180,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,
252457,1,-9.128709e-01,0.0,0.0,9.128709e-01,-15.0,0.0,9.128709e-01,-17.0,0.0,0.912871,-14.0,0.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0,4951.350,47056.275,52641.0,4.275,47056.275,0.000088
256657,2,2.041241e-01,-6.0,0.0,2.041241e-01,-13.0,0.0,2.041241e-01,-12.0,0.0,0.204124,-12.5,0.0,0.204124,-13.0,0.0,-1.020621,-1.5,0.0,49054.770,978570.000,1198062.0,15714.000,978570.000,0.217818
198678,2,4.540312e-08,-20.0,0.0,4.540312e-08,-21.5,0.0,4.540312e-08,-6.5,0.0,-0.000407,-1.0,0.0,0.000204,0.5,0.0,0.000204,-0.5,0.0,98806.050,921721.500,941661.0,13347.000,921721.500,0.193358
310743,1,-4.082483e-01,0.0,0.0,-4.082483e-01,0.0,0.0,-4.082483e-01,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,2.041241,-5.0,0.0,7002.720,67864.230,75024.0,4.230,67864.230,0.000061
394447,2,5.295619e-01,-0.5,0.0,6.115381e-01,-7.0,0.0,-3.183548e-01,-7.0,0.0,-0.229629,-6.5,0.0,-0.180184,-8.5,0.0,-0.412932,0.0,0.0,17141.445,129132.000,129132.0,0.000,129132.000,0.000000


### Zusammenführen und Speichern der Daten

In [63]:
pa_ip = result

In [64]:
pa_ip = pd.merge(pa_ip, cats, how="left", left_index=True, right_index=True)

In [65]:
pa_ip = pd.merge(pa_ip, target, left_index=True, right_index=True) # Nur Kredite die auch in Application enthalten sind werden übernommen
pa_ip = pa_ip.drop(["TARGET"], axis=1)

In [66]:
pa_ip = pa_ip.add_prefix("PA_")

In [67]:
pa_ip.head()

,PA_CNT_CREDITS,PA_IP_PAYMENT_30,PA_IP_DAYS_DIFF_30,PA_IP_AMT_DIFF_30,PA_IP_PAYMENT_60,PA_IP_DAYS_DIFF_60,PA_IP_AMT_DIFF_60,PA_IP_PAYMENT_90,PA_IP_DAYS_DIFF_90,PA_IP_AMT_DIFF_90,PA_IP_PAYMENT_120,PA_IP_DAYS_DIFF_120,PA_IP_AMT_DIFF_120,PA_IP_PAYMENT_150,PA_IP_DAYS_DIFF_150,PA_IP_AMT_DIFF_150,PA_IP_PAYMENT_180,PA_IP_DAYS_DIFF_180,PA_IP_AMT_DIFF_180,PA_AMT_ANNUITY,PA_AMT_APPLICATION,PA_AMT_CREDIT,PA_AMT_DOWN_PAYMENT,PA_AMT_GOODS_PRICE,PA_RATE_DOWN_PAYMENT,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_CODE_REJECT_REASON_HC,PA_CNT_NAME_CLIENT_TYPE_New,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_CNT_NAME_PORTFOLIO_Cards,PA_CNT_NAME_PRODUCT_TYPE_walk-in,PA_CNT_NAME_YIELD_GROUP_high,PA_CNT_NAME_YIELD_GROUP_low_normal,PA_CNT_PRODUCT_COMBINATION_Card Street,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low,PA_CNT_PRODUCT_COMBINATION_POS industry with interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
252457,1,-9.128709e-01,0.0,0.0,9.128709e-01,-15.0,0.0,9.128709e-01,-17.0,0.0,0.912871,-14.0,0.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0,4951.350,47056.275,52641.0,4.275,47056.275,0.000088,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
256657,2,2.041241e-01,-6.0,0.0,2.041241e-01,-13.0,0.0,2.041241e-01,-12.0,0.0,0.204124,-12.5,0.0,0.204124,-13.0,0.0,-1.020621,-1.5,0.0,49054.770,978570.000,1198062.0,15714.000,978570.000,0.217818,2.0,7.0,5.0,1.0,0.0,2.0,5.0,0.0,2.0,1.0,0.0,0.0
198678,2,4.540312e-08,-20.0,0.0,4.540312e-08,-21.5,0.0,4.540312e-08,-6.5,0.0,-0.000407,-1.0,0.0,0.000204,0.5,0.0,0.000204,-0.5,0.0,98806.050,921721.500,941661.0,13347.000,921721.500,0.193358,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
394447,2,5.295619e-01,-0.5,0.0,6.115381e-01,-7.0,0.0,-3.183548e-01,-7.0,0.0,-0.229629,-6.5,0.0,-0.180184,-8.5,0.0,-0.412932,0.0,0.0,17141.445,129132.000,129132.0,0.000,129132.000,0.000000,1.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
156331,2,-7.381905e-01,0.0,0.0,1.416426e-02,0.0,0.0,1.416426e-02,0.0,0.0,-1.114368,-1.0,0.0,1.207845,-12.5,0.0,0.616385,-5.0,0.0,40866.795,793035.000,793035.0,0.000,793035.000,0.000000,2.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0


In [68]:
pa_ip.to_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_ip.csv")

## PA_CC

In [69]:
# pa_cc

In [70]:
n_heads.remove("NAME_TYPE_SUITE")

In [71]:
df = pa_cc
df = df.drop(n_heads + o_heads, axis=1)
df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,CC_BALANCE_30,CC_PAYMENT_30,CC_DPD_30,CC_BALANCE_60,CC_PAYMENT_60,CC_DPD_60,CC_BALANCE_90,CC_PAYMENT_90,CC_DPD_90,CC_BALANCE_120,CC_PAYMENT_120,CC_DPD_120,CC_BALANCE_150,CC_PAYMENT_150,CC_DPD_150,CC_BALANCE_180,CC_PAYMENT_180,CC_DPD_180
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000018,394447,2250.0,45000.0,45000.0,45000.0,-176,0.0,-175.0,-130.0,365243.0,365243.0,365243.0,-1.206367,-1.389108,0.0,1.434002,1.008387,0.0,0.992963,-0.429381,0.0,-0.349513,-0.292196,0.0,-0.415697,-0.215746,0.0,-0.455389,1.318045,0.0
1000030,361282,2250.0,45000.0,45000.0,45000.0,-255,0.0,-243.0,-178.0,365243.0,365243.0,365243.0,0.946004,0.850318,0.0,0.828945,1.109609,0.0,0.861611,0.179508,0.0,-0.393110,-1.267179,0.0,-1.074370,0.280581,0.0,-1.169080,-1.152838,0.0
1000031,131335,2250.0,45000.0,45000.0,45000.0,-499,0.0,-396.0,-362.0,365243.0,365243.0,365243.0,0.722068,-0.427910,0.0,0.719445,-0.427508,0.0,0.791575,-0.445538,0.0,0.204483,-0.215249,0.0,-0.817702,-0.514661,0.0,-1.619869,2.030866,0.0
1000035,436351,2250.0,45000.0,45000.0,45000.0,-178,0.0,365243.0,365243.0,365243.0,365243.0,365243.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0
1000077,181153,2250.0,45000.0,45000.0,45000.0,-351,0.0,365243.0,365243.0,365243.0,365243.0,365243.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0


In [72]:
result = pd.DataFrame(index=pa_cc.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

### Gruppierung

In [73]:
# count
A = df[["SK_ID_CURR"]]
A.reset_index(level=0, inplace=True)
A = A.groupby(by=["SK_ID_CURR"]).count()
A.columns = ["CNT_CREDITS"]
result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Anzahl der Kredite im CC-Datensatz je Kreditnehmer

In [74]:
cc.columns.values

array(['CC_BALANCE_30', 'CC_PAYMENT_30', 'CC_DPD_30', 'CC_BALANCE_60',
       'CC_PAYMENT_60', 'CC_DPD_60', 'CC_BALANCE_90', 'CC_PAYMENT_90',
       'CC_DPD_90', 'CC_BALANCE_120', 'CC_PAYMENT_120', 'CC_DPD_120',
       'CC_BALANCE_150', 'CC_PAYMENT_150', 'CC_DPD_150', 'CC_BALANCE_180',
       'CC_PAYMENT_180', 'CC_DPD_180'], dtype=object)

In [75]:
# means
mean_heads = list(cc.columns.values)

for head in mean_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).mean()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

In [76]:
# sums
sum_heads = ["AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_GOODS_PRICE"]

for head in sum_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).sum()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

In [77]:
result.head()

,CNT_CREDITS,CC_BALANCE_30,CC_PAYMENT_30,CC_DPD_30,CC_BALANCE_60,CC_PAYMENT_60,CC_DPD_60,CC_BALANCE_90,CC_PAYMENT_90,CC_DPD_90,CC_BALANCE_120,CC_PAYMENT_120,CC_DPD_120,CC_BALANCE_150,CC_PAYMENT_150,CC_DPD_150,CC_BALANCE_180,CC_PAYMENT_180,CC_DPD_180,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,
394447,1,-1.206367,-1.389108,0.0,1.434002,1.008387,0.0,0.992963,-0.429381,0.0,-0.349513,-0.292196,0.0,-0.415697,-0.215746,0.0,-0.455389,1.318045,0.0,2250.0,45000.0,45000.0,45000.0
361282,1,0.946004,0.850318,0.0,0.828945,1.109609,0.0,0.861611,0.179508,0.0,-0.393110,-1.267179,0.0,-1.074370,0.280581,0.0,-1.169080,-1.152838,0.0,2250.0,45000.0,45000.0,45000.0
131335,1,0.722068,-0.427910,0.0,0.719445,-0.427508,0.0,0.791575,-0.445538,0.0,0.204483,-0.215249,0.0,-0.817702,-0.514661,0.0,-1.619869,2.030866,0.0,2250.0,45000.0,45000.0,45000.0
436351,1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2250.0,45000.0,45000.0,45000.0
181153,1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2250.0,45000.0,45000.0,45000.0


### Zusammenführung und Speicherung der Daten

In [78]:
pa_cc = result

In [79]:
pa_cc = pd.merge(pa_cc, cats, how="left", left_index=True, right_index=True)

In [80]:
pa_cc = pd.merge(pa_cc, target, left_index=True, right_index=True) # Nur Kredite die auch in Application enthalten sind werden übernommen
pa_cc = pa_cc.drop(["TARGET"], axis=1)

In [81]:
pa_cc = pa_cc.add_prefix("PA_")

In [82]:
pa_cc.head()

,PA_CNT_CREDITS,PA_CC_BALANCE_30,PA_CC_PAYMENT_30,PA_CC_DPD_30,PA_CC_BALANCE_60,PA_CC_PAYMENT_60,PA_CC_DPD_60,PA_CC_BALANCE_90,PA_CC_PAYMENT_90,PA_CC_DPD_90,PA_CC_BALANCE_120,PA_CC_PAYMENT_120,PA_CC_DPD_120,PA_CC_BALANCE_150,PA_CC_PAYMENT_150,PA_CC_DPD_150,PA_CC_BALANCE_180,PA_CC_PAYMENT_180,PA_CC_DPD_180,PA_AMT_ANNUITY,PA_AMT_APPLICATION,PA_AMT_CREDIT,PA_AMT_GOODS_PRICE,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_CODE_REJECT_REASON_HC,PA_CNT_NAME_CLIENT_TYPE_New,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_CNT_NAME_PORTFOLIO_Cards,PA_CNT_NAME_PRODUCT_TYPE_walk-in,PA_CNT_NAME_YIELD_GROUP_high,PA_CNT_NAME_YIELD_GROUP_low_normal,PA_CNT_PRODUCT_COMBINATION_Card Street,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low,PA_CNT_PRODUCT_COMBINATION_POS industry with interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
394447,1,-1.206367,-1.389108,0.0,1.434002,1.008387,0.0,0.992963,-0.429381,0.0,-0.349513,-0.292196,0.0,-0.415697,-0.215746,0.0,-0.455389,1.318045,0.0,2250.0,45000.0,45000.0,45000.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
361282,1,0.946004,0.850318,0.0,0.828945,1.109609,0.0,0.861611,0.179508,0.0,-0.393110,-1.267179,0.0,-1.074370,0.280581,0.0,-1.169080,-1.152838,0.0,2250.0,45000.0,45000.0,45000.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
436351,1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2250.0,45000.0,45000.0,45000.0,1.0,3.0,3.0,0.0,1.0,1.0,1.0,2.0,6.0,1.0,3.0,0.0
181153,1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2250.0,45000.0,45000.0,45000.0,1.0,3.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,0.0,0.0
309691,1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2250.0,45000.0,45000.0,45000.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [83]:
pa_cc.to_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_cc.csv")